In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd, remove, chdir
from os.path import join, basename

sys.path.append(str(Path(getcwd()).parent))
current_dir = getcwd()
print(current_dir)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
from bulletin.systems import Notifica
from bulletin import tables

In [ ]:
nt = Notifica()
nt.load('notifica', compress=False)
nt.df = nt.df.loc[(nt.df['tabagismo']==1) & (nt.df['classificacao_final']==2) & (nt.df['status_notificacao'].isin([1,2])) & (nt.df['excluir_ficha']==2)]
nt.df = nt.df.join(tables['municipios'].set_index('ibge'),on='ibge_residencia')
nt.df = nt.df.loc[nt.df['uf']=='PR']

In [ ]:
nt.check_duplicates(keep='last')
tabagismo = nt.df.drop(index=nt.df.loc[nt.df['duplicated']].index)

In [ ]:
tabagismo['faixa_etaria'] = np.digitize(tabagismo['idade'],[20,30,40,50,60,70,80,90,100],right=False)

In [ ]:
tabagismo.loc[(tabagismo['evolucao']==2), 'evolucao'] = 'Óbito'
tabagismo.loc[(tabagismo['evolucao']!='Óbito'), 'evolucao'] = 'Outra'

In [ ]:
tab_macro_rs_mun = tabagismo.groupby(['macro','rs','regional','ibge_residencia','municipio','evolucao'])[['id']].count().unstack('evolucao').droplevel(axis=1, level=0).fillna(0).astype(int)
tab_macro_rs_mun.columns.name = ''
tab_macro_rs_mun['Caso'] = tab_macro_rs_mun['Óbito'] + tab_macro_rs_mun['Outra']
tab_macro_rs_mun = tab_macro_rs_mun[['Caso','Óbito']]
tab_macro_rs_mun

In [ ]:
tab_macro_rs = tab_macro_rs_mun.groupby(level=['macro','rs','regional']).sum()
tab_macro_rs

In [ ]:
tab_macro= tab_macro_rs.groupby(level=['macro']).sum()
tab_macro

In [ ]:
tab_sexo_fx = tabagismo.groupby(['sexo','faixa_etaria','evolucao'])[['id']].count().unstack('evolucao').droplevel(axis=1,level=0).fillna(0).astype(int)
tab_sexo_fx.columns.name = ''
tab_sexo_fx['Caso'] = tab_sexo_fx['Óbito'] + tab_sexo_fx['Outra']
tab_sexo_fx = tab_sexo_fx[['Caso','Óbito']]
tab_sexo_fx

In [ ]:
tab_sexo = tab_sexo_fx.groupby(level=['sexo']).sum()

In [ ]:
tab_fx = tab_sexo_fx.groupby(level=['faixa_etaria']).sum()

In [ ]:
tab_ano_mes = tabagismo.groupby([tabagismo.rename(columns={'data_diagnostico':'ano'})['ano'].dt.year,tabagismo.rename(columns={'data_diagnostico':'mes'})['mes'].dt.month])[['id']].count().rename(columns={'id':'Caso'})
tab_ano_mes = tab_ano_mes.join(tabagismo.loc[tabagismo['evolucao']=='Óbito'].groupby([tabagismo.rename(columns={'data_cura_obito':'ano'})['ano'].dt.year,tabagismo.rename(columns={'data_cura_obito':'mes'})['mes'].dt.month])[['id']].count().rename(columns={'id':'Óbitos'}))
tab_ano_mes = tab_ano_mes.fillna(0).astype(int)
tab_ano_mes

In [ ]:
writer = pd.ExcelWriter("tabelas_tabagismo.xlsx",
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

In [ ]:
tabagismo[['id','ibge_residencia','sexo','idade','tabagismo','data_diagnostico','evolucao','data_cura_obito','ibge7','rs','regional','macro','municipio','uf','populacao','estado','faixa_etaria']].to_excel(writer,'geral')
tab_macro_rs_mun.reset_index().to_excel(writer,'tab_macro_rs_mun')
tab_macro_rs.reset_index().to_excel(writer,'tab_macro_rs')
tab_macro.reset_index().to_excel(writer,'tab_macro')
tab_sexo_fx.reset_index().to_excel(writer,'tab_sexo_fx')
tab_sexo.reset_index().to_excel(writer,'tab_sexo')
tab_fx.reset_index().to_excel(writer,'tab_fx')
tab_ano_mes.reset_index().to_excel(writer,'tab_ano_mes')

In [ ]:
writer.save()